In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convInput = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1)
        self.conv9 = nn.Conv2d(256, 128, kernel_size=3, stride=1)
        self.conv10 = nn.Conv2d(128, 64, kernel_size=3, stride=1)
        self.conv11 = nn.Conv2d(64, 1, kernel_size=3, stride=1)
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsampling = nn.Upsample(scale_factor=(2,2))

    def forward(self, x):
        out = model(x)
        x = Relu(self.convInput(x), inplace=True)
        x = Relu(self.conv2(x), inplace=True)
        poolX = self.pooling(x)
        
        x2 = Relu(self.conv3(poolX), inplace=True)
        x2 = Relu(self.conv4(x2), inplace=True)
        poolX2 = self.pooling(x2)

        x3 = Relu(self.conv3(poolX2), inplace=True)
        x3 = Relu(self.conv4(x3), inplace=True)
        poolX3 = self.pooling(x3)
        
        x4 = Relu(self.conv5(poolX3), inplace=True)
        x4 = Relu(self.conv6(x4), inplace=True)
        poolX4 = self.pooling(x4)
        
        x5 = Relu(self.conv7(poolX4), inplace=True)
        
        up6 = self.upsampling(x5)
        x6 = Relu(self.conv7(up6), inplace=True)
        x6 = Relu(self.conv8(x6), inplace=True)
        poolX6 = self.pooling(x6)
        merge6 = torch.cat((out,x4,x6),-1)
        x6 = Relu(self.conv8(merge6), inplace=True)
        
        up7 = self.upsampling(x6)
        x7 = Relu(self.conv8(up7), inplace=True)
        x7 = Relu(self.conv8(x7), inplace=True)
        poolX7 = self.pooling(x7)
        merge7 = torch.cat((out,x3,x7),-1)
        x7 = Relu(self.conv8(merge7), inplace=True)
        
        up8 = self.upsampling(x7)
        x8 = Relu(self.conv9(up8), inplace=True)
        x8 = Relu(self.conv6(x8), inplace=True)
        poolX8 = self.pooling(x8)
        merge8 = torch.cat((out,x2,x8),-1)
        x8 = Relu(self.conv6(merge8), inplace=True)
        
        up9 = self.upsampling(x8)
        x9 = Relu(self.conv10(up9), inplace=True)
        x9 = Relu(self.conv4(x9), inplace=True)
        poolX9 = self.pooling(x9)
        merge9 = torch.cat((out,x1,x9),-1)
        x9 = Relu(self.conv4(merge9), inplace=True)
        
        x10 = Relu(self.conv11(x9), inplace=True)
        
        return x10